# Attempting to Get Callbacks from Holoviews Datashader Graph

## Sandbox Example.. nothing in classes yet

In [ ]:
import traitlets as T

import bokeh.models.widgets as bk
import holoviews as hv
import ipycytoscape
import ipywidgets as W
import jupyter_bokeh as jbk
import networkx as nx
from bokeh.models import ColumnDataSource, CustomJS, HoverTool, TapTool
from bokeh.plotting import figure, show
from holoviews import streams
from holoviews.operation.datashader import bundle_graph, datashade, dynspread
from ipyradiant import LoadWidget
from rdflib import BNode, Graph, Literal, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_graph

hv.extension("bokeh")

In [ ]:
def strip_and_produce_rdf_graph(rdf_graph: Graph):
    sparql = """
    CONSTRUCT {
        ?s ?p ?o .
    } 
    WHERE {
        ?s ?p ?o .
        FILTER (!isLiteral(?o)) 
        FILTER (!isLiteral(?s))
    }
    LIMIT 300
    """
    qres = rdf_graph.query(sparql)
    uri_graph = Graph()
    for row in qres:
        uri_graph.add(row)

    new_netx = rdflib_to_networkx_graph(uri_graph)

    original = hv.Graph.from_networkx(new_netx, nx.layout.circular_layout,)
    output_graph = bundle_graph(original)
    return output_graph

In [ ]:
rdf_graph = Graph()
rdf_graph.parse(
    "https://raw.githubusercontent.com/schemaorg/schemaorg/v9.0-release/data/releases/8.0/schema.nt",
    format="nt",
)

output_graph = strip_and_produce_rdf_graph(rdf_graph)

# Create Graph with tooltip involving DataFrame index...

Need to figure out how to get this to produce valuable information.

## This is the best I could get so far, it seems that Selection1D doesn't want to work with this type of graph unfortunately.

In [ ]:
selection_stream = streams.Tap(source=output_graph)

In [ ]:
def labelled_callback(x, y):
    if x is None or y is None:
        return hv.Text(x=0, y=0, text="blank passed in")
    return hv.Text(x=x, y=y, text="testing")


labeller = hv.DynamicMap(labelled_callback, streams=[selection_stream])

In [ ]:
node_tooltips = [
    ("ID", "@index"),
]
node_hover = HoverTool(tooltips=node_tooltips)

Have to use output widget here to maintain interactivity with graph.. however if a
different solution to callbacks is found, might be possible to go back to widget form.

In [ ]:
output = W.Output()

In [ ]:
with output:
    display(
        output_graph.options(
            frame_width=1000,
            frame_height=1000,
            xaxis=None,
            yaxis=None,
            tools=[node_hover, "box_select", "tap"],
        )
    )

In [ ]:
output

We can get the x/y values of where the point was clicked, but I think this is useless
since it doesn't really tell us anything about the graph and they aren't linked to any
points.

In [ ]:
labeller.streams[0].x